In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
W, H = 900, 250  # canvas size

canvas = np.ones((H, W, 3), dtype=np.uint8) * 255  # white background

drawing = False
last_x, last_y = None, None

pen_thickness = 18
pen_color = (0, 0, 0)  # black (BGR)
